In [1]:
!python3 preprocess_data.py -h

usage: preprocess_data.py [-h] -i INPUT -o OUTPUT [-t {gestures,locomotion}]

Preprocess OPPORTUNITY dataset

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        OPPORTUNITY zip file
  -o OUTPUT, --output OUTPUT
                        Processed data file
  -t {gestures,locomotion}, --task {gestures,locomotion}
                        Type of activities to be recognized


In [ ]:
pip install torch

In [ ]:
pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 torchaudio==2.0.2+cu117 --index-url https://download.pytorch.org/whl/cu117

In [ ]:
pip install scikit-learn

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [3]:
# Load the datasets from the provided files
X_train = np.load('./mesl_data/x_train.npy')
X_test = np.load('./mesl_data/x_test.npy')
y_train = np.load('./mesl_data/y_train.npy')
y_test = np.load('./mesl_data/y_test.npy')

# Verify the shapes of the data
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Check the unique values in y_train and y_test
print("Unique values in y_train:", np.unique(y_train))
print("Unique values in y_test:", np.unique(y_test))

# Ensure all labels are within the correct range
assert np.all((y_train >= 0) & (y_train < 26)), "y_train contains out-of-range values"
assert np.all((y_test >= 0) & (y_test < 26)), "y_test contains out-of-range values"

# Normalize data
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    return (data - mean) / std

# Normalize the data
X_train = normalize(X_train)
X_test = normalize(X_test)

# Convert data to float32
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.int64)  # Ensuring labels are in int64
y_test = y_test.astype(np.int64)    # Ensuring labels are in int64

# Reshape data for the model
X_train = X_train.reshape((-1, 1, X_train.shape[1], X_train.shape[2]))
X_test = X_test.reshape((-1, 1, X_test.shape[1], X_test.shape[2]))

print(f"Train data shape after reshape: {X_train.shape}")
print(f"Test data shape after reshape: {X_test.shape}")

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

X_train shape: (4953, 150, 9)
X_test shape: (1320, 150, 9)
y_train shape: (4953,)
y_test shape: (1320,)
Unique values in y_train: [ 0  1  3  5  7  8 11 15 21 23 25]
Unique values in y_test: [ 0  1  3  5  7  8 11 15 21 23 25]
Train data shape after reshape: (4953, 1, 150, 9)
Test data shape after reshape: (1320, 1, 150, 9)
Using device: cuda


In [4]:
# Hardcoded parameters for the new dataset
NB_SENSOR_CHANNELS = 9
NUM_CLASSES = 26  # Updated number of classes
SLIDING_WINDOW_LENGTH = 150
BATCH_SIZE = 16
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128
LEARNING_RATE = 0.0001
NUM_EPOCHS = 150


In [5]:
# Define the network
class DeepConvLSTM(nn.Module):
    def __init__(self):
        super(DeepConvLSTM, self).__init__()
        self.conv1 = nn.Conv2d(1, NUM_FILTERS, (FILTER_SIZE, 1))
        self.conv2 = nn.Conv2d(NUM_FILTERS, NUM_FILTERS, (FILTER_SIZE, 1))
        self.conv3 = nn.Conv2d(NUM_FILTERS, NUM_FILTERS, (FILTER_SIZE, 1))
        self.conv4 = nn.Conv2d(NUM_FILTERS, NUM_FILTERS, (FILTER_SIZE, 1))
        self.lstm1 = nn.LSTM(NUM_FILTERS * NB_SENSOR_CHANNELS, NUM_UNITS_LSTM, batch_first=True)
        self.lstm2 = nn.LSTM(NUM_UNITS_LSTM, NUM_UNITS_LSTM, batch_first=True)
        self.fc = nn.Linear(NUM_UNITS_LSTM, NUM_CLASSES)

        # Weight initialization
        nn.init.kaiming_uniform_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv3.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv4.weight, nonlinearity='relu')
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = x.permute(0, 2, 1, 3).contiguous().view(x.size(0), x.size(2), -1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = self.fc(x[:, -1, :])
        return x

In [6]:
# Create datasets and dataloaders
train_data = torch.utils.data.TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_data = torch.utils.data.TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# Initialize the model, loss function and optimizer
try:
    model = DeepConvLSTM().to(device)
    print("Model initialized and moved to GPU")
except RuntimeError as e:
    print(f"Error initializing model: {e}")
    raise

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

Model initialized and moved to GPU


In [7]:
# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {train_loss / len(train_loader)}")


Epoch 1/150, Loss: 1.4973685215557775
Epoch 2/150, Loss: 0.45932051899933046
Epoch 3/150, Loss: 0.3200265549243458
Epoch 4/150, Loss: 0.2558509041525183
Epoch 5/150, Loss: 0.2178185812437967
Epoch 6/150, Loss: 0.19725385060113285
Epoch 7/150, Loss: 0.17581168552259765
Epoch 8/150, Loss: 0.15966959931407004
Epoch 9/150, Loss: 0.1347184813356087
Epoch 10/150, Loss: 0.1358865763316111
Epoch 11/150, Loss: 0.1201759201985213
Epoch 12/150, Loss: 0.10988826413315932
Epoch 13/150, Loss: 0.08893329867691312
Epoch 14/150, Loss: 0.07892573440725344
Epoch 15/150, Loss: 0.05899791966864629
Epoch 16/150, Loss: 0.06870723616185358
Epoch 17/150, Loss: 0.053356911824460354
Epoch 18/150, Loss: 0.05147168390328936
Epoch 19/150, Loss: 0.04560612672949285
Epoch 20/150, Loss: 0.07167865965034692
Epoch 21/150, Loss: 0.04812297323031441
Epoch 22/150, Loss: 0.04207820947819029
Epoch 23/150, Loss: 0.028376454530159884
Epoch 24/150, Loss: 0.047013506107595596
Epoch 25/150, Loss: 0.0356696921185733
Epoch 26/150, 

In [8]:
# Save the model parameters
torch.save(model.state_dict(), 'weights/DeepConvLSTM_trained_mesl_data.pth')



In [12]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
device = torch.device('cpu')
model.to(device)
# Evaluation
model.eval()
test_pred = []
test_true = []
batch_times = []
sample_times = []

with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to('cpu'), targets.to('cpu')

        # Start time for the batch
        start_time = time.time()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # End time for the batch
        end_time = time.time()

        # Record the inference time for the batch
        batch_inference_time = end_time - start_time
        batch_times.append(batch_inference_time)

        # Calculate per-sample time for the batch
        batch_size = inputs.size(0)
        per_sample_time = batch_inference_time / batch_size
        sample_times.append(per_sample_time)

        test_pred.extend(preds.cpu().numpy())
        test_true.extend(targets.cpu().numpy())

        # Print per-batch time and per-sample time for the current batch
        #print(f"Batch {batch_idx + 1}:")
        #print(f"  Batch Inference Time: {batch_inference_time:.4f} seconds")
        #print(f"  Per-Sample Inference Time: {per_sample_time:.8f} seconds")

# Calculate metrics
accuracy = accuracy_score(test_true, test_pred)
macro_precision = precision_score(test_true, test_pred, average='macro')
macro_recall = recall_score(test_true, test_pred, average='macro')
macro_f1 = f1_score(test_true, test_pred, average='macro')

# Calculate average batch time and average per-sample time
average_batch_time = sum(batch_times) / len(batch_times)
average_per_sample_time = sum(sample_times) / len(sample_times)

# Results presentation
print(f"\nOverall Test Results:")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")
print(f"Macro F1-score: {macro_f1:.4f}")
print(f"Average Batch Inference Time: {average_batch_time:.4f} seconds")
print(f"Average Per-Sample Inference Time: {average_per_sample_time:.8f} seconds")



Overall Test Results:
Test Accuracy: 0.9053
Macro Precision: 0.9144
Macro Recall: 0.9053
Macro F1-score: 0.9010
Average Batch Inference Time: 0.0084 seconds
Average Per-Sample Inference Time: 0.00053201 seconds


In [10]:
# Calculate metrics
accuracy = accuracy_score(test_true, test_pred)
macro_precision = precision_score(test_true, test_pred, average='macro')
macro_recall = recall_score(test_true, test_pred, average='macro')
macro_f1 = f1_score(test_true, test_pred, average='macro')

# Results presentation
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")
print(f"Macro F1-score: {macro_f1:.4f}")

Test Accuracy: 0.9053
Macro Precision: 0.9144
Macro Recall: 0.9053
Macro F1-score: 0.9010
